# Vanderbeis Local Warming Model

For this to solve, you'll need to install the Clp package. Canvas has directions for installing packages


In [40]:
include("vanerbei_model.jl")

m = run_vanerbei_model()

A JuMP Model
Minimization problem with:
Variables: 45408
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 22701 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 45402 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 22701 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Ipopt
Names registered in the model: DEV, T, X0, X1, X2, X3, X4, X5, devlb, devub, tdef

Using GamsStructure, much slower. Improvements to come.

In [ ]:
using JuMP
using Clp
using GamsStructure
using NamedArrays

using Ipopt


using CSV

In [2]:
GU = load_universe("julia_data")

Sets

d => Dates for which we have temperature reports

Parameters

ave => (:d,) => Average temperatures
day => (:d,) => Assignment of dates to days


Build and optimize the model. This is taking a long time and I'm not certain why. (julia version 1.8)

Probably my fault. NamedArrays build the model in .4s vs 1m24s

In [29]:
m = Model(Ipopt.Optimizer)

D = [d for d∈GU[:d]]

day = GU[:day]
ave = GU[:ave]

@variables(m,begin
    DEV[GU[:d]] >= 0 #Deviation
    T[GU[:d]]
    X0
    X1
    X2
    X3
    X4
    X5
end)

@objective(m,Min, sum(DEV[d] for d∈GU[:d]))

@constraint(m,tdef[d∈GU[:d]],
    T[d] == X0 + X1*day[[d]] + X2*cos(2*pi*day[[d]]/365.25)
                             + X3*sin(2*pi*day[[d]]/365.25)
                             + X4*cos(2*pi*day[[d]]/(10.7*365.25))
                             + X5*sin(2*pi*day[[d]]/(10.7*365.25))
)

@constraint(m,devlb[d∈GU[:d]],
    DEV[d] >= ave[[d]] - T[d]
)

@constraint(m,devub[d∈GU[:d]],
    DEV[d] >= T[d] - ave[[d]]
);

In [18]:
optimize!(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:   158907
Number of nonzeros in inequality constraint Jacobian.:    90804
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    45408
                     variables with only lower bounds:    22701
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    22701
Total number of inequality c

Get the last 5 variables and display them.

In [21]:
for var in all_variables(m)[end-5:end]
    println("$var => $(value(var))")
end

X0 => 46.11842606520766
X1 => 8.79093173445956e-5
X2 => -23.836946818497402
X3 => -8.077121320465762
X4 => -0.2425550775450732
X5 => 0.12641036429077987


In [22]:
println("Warming = $(value(m[:X1])*365.25*100)")

Warming = 3.210887816011354


Recreating the model using NamedArrays to see if speed improves

In [23]:
D = [d for d∈GU[:d]]
day = NamedArray(GU[:day][:d],D)
ave = NamedArray(GU[:ave][:d],D);

In [26]:
m2 = Model(Ipopt.Optimizer)


@variables(m2,begin
    DEV[D] >= 0 #Deviation
    T[D]
    X0
    X1
    X2
    X3
    X4
    X5
end)

@objective(m2,Min, sum(DEV[d] for d∈D))

@constraint(m2,tdef[d∈D],
    T[d] == X0 + X1*day[d] + X2*cos(2*pi*day[d]/365.25)
                             + X3*sin(2*pi*day[d]/365.25)
                             + X4*cos(2*pi*day[d]/(10.7*365.25))
                             + X5*sin(2*pi*day[d]/(10.7*365.25))
)

@constraint(m2,devlb[d∈D],
    DEV[d] >= ave[d] - T[d]
)

@constraint(m2,devub[d∈D],
    DEV[d] >= T[d] - ave[d]
);

In [28]:
optimize!(m2)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:   158907
Number of nonzeros in inequality constraint Jacobian.:    90804
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    45408
                     variables with only lower bounds:    22701
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    22701
Total number of inequality constraints...............:    45402
        inequality constraints with only lower bounds:    45402
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.2700977e+02 9.09e+01 6.67e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 